In [3]:
import vtk

def extract_centroids_from_glyphs(input_vtp, output_csv):
    # Load the merged glyph mesh
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(input_vtp)
    reader.Update()
    data = reader.GetOutput()

    # Use connectivity filter to group separate glyphs (spheres)
    conn_filter = vtk.vtkConnectivityFilter()
    conn_filter.SetInputData(data)
    conn_filter.SetExtractionModeToAllRegions()
    conn_filter.ColorRegionsOn()
    conn_filter.Update()

    labeled_data = conn_filter.GetOutput()
    region_array = labeled_data.GetCellData().GetArray("RegionId")
    num_regions = conn_filter.GetNumberOfExtractedRegions()

    # Map region ID → list of point IDs
    region_point_ids = {i: set() for i in range(num_regions)}

    for cell_id in range(labeled_data.GetNumberOfCells()):
        region_id = int(region_array.GetTuple1(cell_id))
        pt_ids = vtk.vtkIdList()
        labeled_data.GetCellPoints(cell_id, pt_ids)
        for j in range(pt_ids.GetNumberOfIds()):
            pid = pt_ids.GetId(j)
            region_point_ids[region_id].add(pid)

    # Compute centroids
    points = labeled_data.GetPoints()
    centroids = []
    for pid_list in region_point_ids.values():
        coords = [points.GetPoint(pid) for pid in pid_list]
        x, y, z = map(lambda c: sum(c)/len(c), zip(*coords))
        centroids.append((x, y, z))

    # Write to CSV
    with open(output_csv, "w") as f:
        f.write("x,y,z\n")
        for x, y, z in centroids:
            f.write(f"{x},{y},{z}\n")

    print(f"✅ Extracted {len(centroids)} sphere centers to: {output_csv}")


In [5]:
extract_centroids_from_glyphs("FaultSurface1.vtp", "FaultSurface1.csv")
extract_centroids_from_glyphs("FaultSurface2.vtp", "FaultSurface2.csv")
extract_centroids_from_glyphs("FaultSurface3.vtp", "FaultSurface3.csv")

✅ Extracted 21 sphere centers to: FaultSurface1.csv
✅ Extracted 33 sphere centers to: FaultSurface2.csv
✅ Extracted 27 sphere centers to: FaultSurface3.csv


In [6]:
import numpy as np
from scipy.spatial import Delaunay

def write_geo_from_surface_points(csv_path, output_geo="surface_from_points.geo", mesh_size=5.0):
    # Load (x, y, z) points
    points = np.loadtxt(csv_path, delimiter=",", skiprows=1)
    xy = points[:, :2]
    z = points[:, 2]

    # 2D Delaunay triangulation in (x, y)
    tri = Delaunay(xy)

    with open(output_geo, "w") as f:
        f.write(f"// Gmsh .geo file generated from {csv_path}\n\n")

        # Write points
        for i, (x, y, z_val) in enumerate(points):
            f.write(f"Point({i+1}) = {{{x}, {y}, {z_val}, {mesh_size}}};\n")

        f.write("\n")

        # Write triangles as discrete surface facets
        surface_ids = []
        for j, simplex in enumerate(tri.simplices):
            ids = [i+1 for i in simplex]
            f.write(f"Triangle({j+1}) = {{{ids[0]}, {ids[1]}, {ids[2]}}};\n")
            surface_ids.append(j+1)

        f.write("\n// Define surface from triangle mesh\n")
        f.write("Surface Loop(1) = {")
        f.write(", ".join(str(sid) for sid in surface_ids))
        f.write("};\n")
        f.write("Surface(1) = {1};\n")

    print(f"✅ Wrote Gmsh .geo file to: {output_geo}")


In [7]:
write_geo_from_surface_points("FaultSurface1.csv", "FaultSurface1.geo", mesh_size=2.0)
write_geo_from_surface_points("FaultSurface2.csv", "FaultSurface2.geo", mesh_size=2.0)
write_geo_from_surface_points("FaultSurface3.csv", "FaultSurface3.geo", mesh_size=2.0)

✅ Wrote Gmsh .geo file to: FaultSurface1.geo
✅ Wrote Gmsh .geo file to: FaultSurface2.geo
✅ Wrote Gmsh .geo file to: FaultSurface3.geo
